In [6]:
from au2v import toydata

toydata.generate_toydata("toydata-paper")

(                                                           sequence
 user_id                                                            
 u_0_M_20_50_M1    v_1_E_2000 v_1_E_2000 v_1_M_1960 v_1_M_1990 v_...
 u_1_M_20_50_M2    v_0_E_2000 v_2_M_2000 v_3_M_2000 v_4_E_1960 v_...
 u_2_M_20_50_M3    v_0_M_2000 v_1_M_1960 v_1_M_2000 v_1_E_2000 v_...
 u_3_M_20_50_M4    v_0_E_2000 v_0_E_2000 v_0_M_2000 v_2_M_2000 v_...
 u_4_M_20_50_M5    v_0_M_1980 v_1_M_1980 v_2_M_1970 v_3_M_2000 v_...
 ...                                                             ...
 u_995_F_60_50_F1  v_0_F_1980 v_1_F_1960 v_1_F_1960 v_3_F_1960 v_...
 u_996_F_60_50_F2  v_0_F_1960 v_0_F_1960 v_1_M_1970 v_2_F_2000 v_...
 u_997_F_60_50_F3  v_1_E_1960 v_2_F_2000 v_3_F_1960 v_4_F_1960 v_...
 u_998_F_60_50_F4  v_0_M_1990 v_1_E_1960 v_2_F_1990 v_3_F_1960 v_...
 u_999_F_60_50_F5  v_2_E_1970 v_4_F_1990 v_5_M_1960 v_5_F_1970 v_...
 
 [10000 rows x 1 columns],
                                                            sequence
 user